In [19]:
# Import necessary dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.utils.class_weight import compute_class_weight
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, f1_score
from google.colab import drive

In [2]:
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Step 1: Load and preview the dataset
invoice_df = pd.read_csv("/content/drive/MyDrive/Cleaned_Invoice_4.csv", encoding='latin1').iloc[:, :14]
invoice_df.head()

,Vendor,Month,Year,PO#,Invoice Date,Invoice#,total Invoice Amt,TRS,Project,Amount,Cost Centre,Order#,Dept
0,vendor 1039,Apr,2021,4501190248,2021-04-30,4,"$18,850.00",SDS7089109,WSH LINK Replacement - D&I,$455.00,165239,300007147,FIN
1,vendor 1039,Apr,2021,4501190248,2021-04-30,4,"$18,850.00",SDS7090144,AIMS Replacement (PDO-217),"$3,575.00",171318,930089792,BTT
2,vendor 1039,Apr,2021,4501190248,2021-04-30,4,"$18,850.00",SDS7089063,Petrinex - D&I,$877.50,190230,300007150,ARD
3,vendor 1039,Apr,2021,4501190248,2021-04-30,4,"$18,850.00",SDS7090248,M3P Vitality Upgrade - Implementation Phase - ...,"$4,062.50",195006,300007300,JUS
4,vendor 1039,Apr,2021,4501180577,2021-04-30,5,"$18,850.00",SDS7089423,Civil Legal Billing System - Scoping,"$5,005.00",171318,300007265,JUS


In [4]:
# Step 2: Extract beneficial columns
inv_data = invoice_df[['Vendor', 'PO#', 'TRS', 'Order#', 'Dept', 'Cost Centre']]

In [5]:
# Step 3: One-hot encode categorical features, except target ('Cost Centre')
dummies_df = pd.get_dummies(inv_data, columns=["Vendor", "PO#", "TRS", "Order#", "Dept"])

In [6]:
# Step 4: Separate the features (X) and target (y)
y = dummies_df["Cost Centre"].values
X = dummies_df.drop(columns="Cost Centre").values

In [7]:
# Step 5: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
#  Step 6: Standardize (scale) the feature data
scaler = StandardScaler()

# Fit on training data, transform both training and test sets
X_train_scaled = scaler.fit_transform(X_train).astype(np.float32)
X_test_scaled = scaler.transform(X_test).astype(np.float32)

In [11]:
# Step 7: One-hot encode the target labels for multi-class classification
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
y_train_encoded = encoder.fit_transform(y_train.reshape(-1, 1))
y_test_encoded = encoder.transform(y_test.reshape(-1, 1))


In [12]:
# Step 8: Define the neural network model
number_input_features = X_train_scaled.shape[1]  # Number of features (input size)
num_classes = y_train_encoded.shape[1]  # Number of output classes (73 classes)

In [13]:
# Create a sequential model
nn = models.Sequential()

# Input layer with 128 neurons
nn.add(layers.Dense(128, activation='relu', input_dim=number_input_features))

# Hidden layer with 64 neurons
nn.add(layers.Dense(64, activation='relu'))

# Hidden layer with 32 neurons
nn.add(layers.Dense(32, activation='relu'))

# Output layer with 73 neurons (one per class), using softmax for multi-class classification
nn.add(layers.Dense(num_classes, activation='softmax'))

# Step 9: Compile the model with categorical crossentropy loss for multi-class classification
nn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
nn.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 128)                 │         115,840 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 72)                  │           2,376 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 128,552 (502.16 KB)

 Trainable params: 128,552 (502.16 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
# Step 10: Compute class weights to handle class imbalance in the target variable
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(enumerate(class_weights))

In [15]:
# Step 11: Train the model with class weights
history = nn.fit(X_train_scaled, y_train_encoded,
                 epochs=100,  # Set the number of epochs
                 batch_size=32,  # Set the batch size
                 class_weight=class_weight_dict,  # Pass the computed class weights
                 validation_data=(X_test_scaled, y_test_encoded),  # Use the test set for validation
                 verbose=1)

Epoch 1/100
182/182 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.2111 - loss: 3.4923 - val_accuracy: 0.9224 - val_loss: 0.8743
Epoch 2/100
182/182 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9688 - loss: 0.0873 - val_accuracy: 0.9897 - val_loss: 0.1493
Epoch 3/100
182/182 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9964 - loss: 0.0123 - val_accuracy: 0.9938 - val_loss: 0.0393
Epoch 4/100
182/182 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 1.0000 - loss: 0.0038 - val_accuracy: 0.9938 - val_loss: 0.0252
Epoch 5/100
182/182 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9998 - loss: 0.0023 - val_accuracy: 0.9945 - val_loss: 0.0189
Epoch 6/100
182/182 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9998 - loss: 0.0016 - val_accuracy: 0.9952 - val_loss: 0.0156
Epoch 7/100
182/182 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9998 - loss: 0.0011 - val_accuracy: 0.9952 - val_loss: 0.0139
Epoch 8/100
182/182 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 1.0000 - loss: 7.9711e-04 - val_

In [21]:
#  Evaluate the Model
test_loss, test_accuracy = nn.evaluate(X_test_scaled, y_test_encoded, verbose=0)
print(f"Loss: {test_loss}, Accuracy: {test_accuracy}")

# Make predictions on the test set (outputs probabilities, so get the class index)
y_pred_probs = nn.predict(X_test_scaled)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test_encoded, axis=1)

# Evaluate the Model
accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Classification report: includes precision, recall, and F1-score for each class
print("Classification Report:")
print(classification_report(y_true, y_pred, zero_division=1))

# Calculate the F1-score (macro, micro, or weighted depending on your needs)
f1 = f1_score(y_true, y_pred, average='weighted')
print(f"F1 Score (Weighted): {f1:.4f}")

# Confusion Matrix
confusion = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(confusion)


Loss: 0.01599120907485485, Accuracy: 0.9931318759918213
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
Accuracy: 0.9931
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.80      0.89        10
           1       1.00      1.00      1.00        13
           3       1.00      1.00      1.00        15
           4       1.00      1.00      1.00        21
           5       1.00      1.00      1.00        12
           6       1.00      1.00      1.00         5
           7       1.00      1.00      1.00        13
           8       1.00      1.00      1.00         7
           9       1.00      1.00      1.00        15
          10       1.00      1.00      1.00        19
          11       1.00      1.00      1.00         4
          12       1.00      1.00      1.00         2
          14       1.00      1.00      1.00         3
          15       0.67      1.00      0.80         4
          16       1.00      1.00      1.00        10
